In [ ]:
"""This Script compares the data from the i3 dataset with the data from LinkedIn. Crunchbase data needs to be acquired directly from i3."""

In [ ]:
### Import data
import pandas as pd

# import the Crunchbase data; the data has to be acquired from Crunchbase directly
directory = 'Path to i3 company data'
df_i3 = pd.read_csv(directory)

# Import the LinkedIn data
df = pd.read_csv("../Data/LinkedIn_Orgs_with_Websites.csv", index=False)

In [ ]:
"""Preprocessing"""
# only take Sector tags from i3 that are relevant for climatetech (our technology scope)
relevant_climatetech= ['solar','energy storage', 'wind' ,
 'nuclear fusion' ,'hydrogen', 'biomass & waste-to-energy', 'nuclear fission','electric vehicle (ev) charging','rail'
, 'geothermal', 'hydro & marine','fuel cells']

df_i3_climatetech = df_i3[df_i3['Sector Tag'].isin(relevant_climatetech)]
print(df_i3_climatetech.shape[0]) # yields 6964 companies

# ensure that the i3 data only contains companies that are still active (i.e., not acquired, closed)
df_i3_climatetech = df_i3_climatetech[df_i3_climatetech.Status.isin(["Private","Public"])]
print(df_i3_climatetech.shape[0]) # yields 5179

# take only companies that were updated after 2020, as some companies do not exist anymore, without indication in the i3 dataset
df_i3_climatetech['Updated On'] = pd.to_datetime(df_i3_climatetech['Updated On'])
df_i3_climatetech = df_i3_climatetech[df_i3_climatetech['Updated On'] > '2020-01-01']
print(df_i3_climatetech.shape[0]) # yields 5104


In [ ]:
 #Functions to create keys for merging
 
import re
def extract_domain_body(url):
    # Extract the domain name while ignoring subdomains like 'www', 'en', 'co', etc.
    match = re.search(r'https?://(?:[a-zA-Z0-9-]+\.)*?([a-zA-Z0-9-]+)\.(?:[a-z]{2,})', url)
    return match.group(1) if match else None

def extract_domain_body_from_df(df, column):
    df['address'] = df[column].str.replace("www.","").str.replace("/en.","/").str.replace("/new.","/").apply(lambda x: extract_domain_body(x) if pd.notna(x) else None)
    return df

In [ ]:
# i3 companies    
df_i3_climatetech = extract_domain_body_from_df(df_i3_climatetech, "Website")
df_i3_climatetech['address'] = df_i3_climatetech['address'].str.lower()

# linkedin companies
df = extract_domain_body_from_df(df, "website")
df.address = df.address.str.lower()

In [ ]:
"""Merge the two datasets """

import numpy as np
import pandas as pd
import re

df_i3 = df_i3_climatetech.copy()

# 1. Exact match on address
list_addresses = df['address'].dropna().unique().tolist()

df_i3["matched"] = df_i3["address"].isin(list_addresses).astype(int)

print(f"Exact address matches found: {df_i3['matched'].sum()}")

# 2. Map Linkedin names based on matched addresses
df_unique = df.drop_duplicates(subset=["address"])
address_to_linkedin = df_unique.set_index("address")["Linkedin_name"].to_dict()

df_i3["Linkedin_name"] = df_i3.loc[df_i3["matched"] == 1, "address"].map(address_to_linkedin)

# 3. Prepare matching on names
df["stem"] = df["name"].dropna().str.lower().apply(lambda x: re.split(r'[\s:-]', x)[0])
df["Linkedin_name_clean"] = df["Linkedin_name"].str.replace("-", " ", regex=False).str.lower()

df_i3["stem"] = df_i3["Company"].str.lower().apply(lambda x: re.split(r'[\s:-]', x)[0])


# 4. Fuzzy matching loop
errors = []

for i, row in df_i3.iterrows():
    if row["matched"] == 1:
        continue  # Already matched on address

    name = str(row["Company"]).lower()
    country = row.get("country_code", None)

    while True:
        try:
            # Direct name contains match
            matches = df[df['name'].str.lower().str.contains(name, na=False)]

            # If no matches, try clean LinkedIn name
            if matches.empty:
                matches = df[df['Linkedin_name_clean'].str.contains(name, na=False)]

            # If still nothing and name has multiple words, try last word
            if matches.empty and len(re.split(r'[\s:-]', name)) > 1:
                name = re.split(r'[\s:-]', name)[-1]
                continue

            # If still nothing, give up
            if matches.empty:
                errors.append(name)
                break

            # If too many matches, try filtering by country
            if matches.shape[0] > 5 and country:
                matches = matches[matches['country'] == country]

            if not matches.empty:
                df_i3.loc[i, "matched"] = 1
            break

        except Exception as e:
            print(f"Error at row {i}: {e}")
            errors.append(name)
            break

    if i % 100 == 0:
        print(f"Processed {i} rows")

print(f"Total matches found: {df_i3['matched'].sum()}")


In [ ]:
# Descriptives
print("Number of matched companies",df_i3[(df_i3['matched'] == 1) ].shape[0]) # yields 4406
print("Number of relevant companies from i3", df_i3.shape[0]) # yields 5104
print("Share of matches: ", df_i3[(df_i3['matched'] == 1) ].shape[0] / df_i3.shape[0]) # yields 0.863

In [ ]:
# Analysis across countries
# create a subdataset grouping mean of the matches across countries
df_i3_grouped = df_i3[['Country', 'matched','Company']].groupby(['Country']).agg({'matched_any': 'mean','Company':'count'}).reset_index()

#rename to Mean and Count Total
df_i3_grouped = df_i3_grouped.rename(columns={'matched': 'Mean', 'Company': 'Count Total'})

# sort by Count Total
df_i3_grouped = df_i3_grouped.sort_values(by='Count Total', ascending=False)

print(df_i3_grouped)

# save the results
df_i3_grouped.to_csv("countries_matched_i3.csv", index=False)

In [ ]:
# Across Development Stages

df_i3_grouped = df_i3[['Development Stage', 'matched','Company']].groupby(['Development Stage']).agg({'matched_any': 'mean','Company':'count'}).reset_index()

#rename to Mean and Count Total
df_i3_grouped = df_i3_grouped.rename(columns={'matched': 'Mean', 'Company': 'Count Total'})

# sort by Count Total
df_i3_grouped = df_i3_grouped.sort_values(by='Count Total', ascending=False)
print(df_i3_grouped)
# save the results
df_i3_grouped.to_csv("development_stage_matched_i3.csv", index=False)

In [ ]:
# Across Sector Tags
df_i3_grouped = df_i3[['Sector Tag', 'matched','Company']].groupby(['Sector Tag']).agg({'matched_any': 'mean','Company':'count'}).reset_index()
#rename to Mean and Count Total

df_i3_grouped = df_i3_grouped.rename(columns={'matched': 'Mean', 'Company': 'Count Total'})
# sort by Count Total
df_i3_grouped = df_i3_grouped.sort_values(by='Count Total', ascending=False)
print(df_i3_grouped)
# save the results
df_i3_grouped.to_csv("primary_sector_matched_i3.csv", index=False)

In [ ]:
# by Status (Public vs. Private)
df_i3_grouped = df_i3[['Status', 'matched','Company']].groupby(['Status']).agg({'matched_any': 'mean','Company':'count'}).reset_index()
#rename to Mean and Count Total
df_i3_grouped = df_i3_grouped.rename(columns={'matched': 'Mean', 'Company': 'Count Total'})
# sort by Count Total
df_i3_grouped = df_i3_grouped.sort_values(by='Count Total', ascending=False)
print(df_i3_grouped)
# save the results
df_i3_grouped.to_csv("status_matched_i3.csv", index=False)